### Stock Analysis

In this part we will make analysis using RSI, volatility and other basic financial tools.

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

Creating a function to retrieve financial data from yfinance.

In [2]:
def get_stock_data(ticker, period='1y'):
    df = yf.download(ticker, period=period, auto_adjust=True)
    
    # Sütun isimlerindeki karmaşıklığı (MultiIndex) temizle
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(1)
    
    # Getirileri Hesapla
    # Log Getiri (Analiz ve Volatilite için daha doğru)
    df['Log_Return'] = np.log(df['Close'] / df['Close'].shift(1))
    
    return df

Creating a function that generates technical indicators.

In [3]:
def add_indicators(df):
    # Simple Moving Average (SMA 20 and 50)
    df['SMA_20'] = df['Close'].rolling(window=20).mean()
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    
    # RSI (Relative Strength Index)
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    
    return df

Creating a function to generate plot-candlestick

In [4]:
def plot_dashboard(df, ticker):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                        vertical_spacing=0.03, subplot_titles=(f'{ticker} Price Analysis', 'RSI Momentum'),
                        row_width=[0.2, 0.7])

    # Candlestick Graph
    fig.add_trace(go.Candlestick(x=df.index,
                open=df['Open'], high=df['High'],
                low=df['Low'], close=df['Close'], name='Fiyat'), row=1, col=1)

    # SMA Lines
    fig.add_trace(go.Scatter(x=df.index, y=df['SMA_20'], line=dict(color='orange', width=1), name='SMA 20'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['SMA_50'], line=dict(color='blue', width=1), name='SMA 50'), row=1, col=1)

    # 2nd Row: RSI
    fig.add_trace(go.Scatter(x=df.index, y=df['RSI'], line=dict(color='purple', width=2), name='RSI'), row=2, col=1)
    
    # Reference lines for RSI at 70 and 30
    fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)
    fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1)

    # Layout adjustments
    fig.update_layout(title=f'{ticker} Technical Analysis Dashboard', yaxis_title='Price', xaxis_rangeslider_visible=False, height=800)
    fig.show()

## Using the functions:

In [5]:
symbol = "GARAN.IS" # Or "AAPL", "NVDA"

# 1. Get Data
data = get_stock_data(symbol)
print(data.head())

# 2. Add Indicators
data = add_indicators(data)

# 3. Plot Dashboard
plot_dashboard(data, symbol)

# 4. Show Last 5 Days
print(data[['Close', 'Log_Return', 'RSI']].tail())

[*********************100%***********************]  1 of 1 completed


Price            Close        High         Low        Open    Volume  \
Date                                                                   
2024-12-12  121.314445  123.243133  120.350102  122.375222  18260905   
2024-12-13  123.243134  123.243134  119.192885  120.060796  17103441   
2024-12-16  123.243134  125.075381  122.085917  123.243134  21133440   
2024-12-17  125.846855  126.811199  122.278786  123.532436  22313133   
2024-12-18  123.821732  126.811203  123.821732  125.268247  19049800   

Price       Log_Return  
Date                    
2024-12-12         NaN  
2024-12-13    0.015773  
2024-12-16    0.000000  
2024-12-17    0.020907  
2024-12-18   -0.016223  


Price            Close  Log_Return        RSI
Date                                         
2025-12-08  141.199997   -0.002122  71.428548
2025-12-09  140.600006   -0.004258  65.873008
2025-12-10  139.699997   -0.006422  59.051707
2025-12-11  139.199997   -0.003586  64.018676
2025-12-12  140.399994    0.008584  62.801895


## Volatility


In [6]:
def volatility_analysis(df):
    volatility = df['Log_Return'].rolling(window=21).std() * np.sqrt(252)  # Annualized Volatility
    df['Volatility'] = volatility
    return df

def year_volality_analysis(df):
    yearly_volatility = df['Log_Return'].resample('Y').std() * np.sqrt(252)
    df['Yearly_Volatility'] = yearly_volatility
    return df

# Volatility Analysis for multiple symbols
symbol1 = "NVDA"
data1 = get_stock_data(symbol1)
data1 = add_indicators(data1)

# Volatility Analysis for symbol1
data1 = volatility_analysis(data1)
print(f"Volatility Analysis for {symbol1}:")
print(data1[['Close', 'Log_Return', 'Volatility', 'RSI']].tail())

symbol2 = "AAPL"
data2 = get_stock_data(symbol2)
data2 = add_indicators(data2)

# Volatility Analysis for symbol2
data2 = volatility_analysis(data2)
print(f"Volatility Analysis for {symbol2}:")
print(data2[['Close', 'Log_Return', 'Volatility', 'RSI']].tail())

symbol3 = "GARAN.IS"
data3 = get_stock_data(symbol3)
data3 = add_indicators(data3)
# Volatility Analysis for symbol3
data3 = volatility_analysis(data3)
print(f"Volatility Analysis for {symbol3}:")
print(data3[['Close', 'Log_Return', 'Volatility', 'RSI']].tail())

[*********************100%***********************]  1 of 1 completed

Volatility Analysis for NVDA:
Price            Close  Log_Return  Volatility        RSI
Date                                                     
2025-12-08  185.550003    0.017067    0.382635  48.879194
2025-12-09  184.970001   -0.003131    0.382711  54.338761
2025-12-10  183.779999   -0.006454    0.321538  46.384468
2025-12-11  180.929993   -0.015629    0.310405  50.432113
2025-12-12  175.020004   -0.033210    0.326260  45.047403



[*********************100%***********************]  1 of 1 completed


Volatility Analysis for AAPL:
Price            Close  Log_Return  Volatility        RSI
Date                                                     
2025-12-08  277.890015   -0.003198    0.167281  66.456347
2025-12-09  277.179993   -0.002558    0.166387  65.040131
2025-12-10  278.779999    0.005756    0.166714  65.550816
2025-12-11  278.029999   -0.002694    0.150891  68.817878
2025-12-12  278.279999    0.000899    0.148748  62.903841


[*********************100%***********************]  1 of 1 completed

Volatility Analysis for GARAN.IS:
Price            Close  Log_Return  Volatility        RSI
Date                                                     
2025-12-08  141.199997   -0.002122    0.268843  71.428548
2025-12-09  140.600006   -0.004258    0.249783  65.873008
2025-12-10  139.699997   -0.006422    0.239410  59.051707
2025-12-11  139.199997   -0.003586    0.239618  64.018676
2025-12-12  140.399994    0.008584    0.240230  62.801895


## Package

In [7]:
# Creating packaged analysis function
def analysis(symbol, period='1y'):
    data = get_stock_data(symbol, period=period)
    data = add_indicators(data)
    data = volatility_analysis(data)
    return data

AAPL_data = analysis("AAPL", "10y")
print(AAPL_data[['Close', 'Log_Return', 'Volatility', 'RSI']].tail())
AAPL_data.to_csv("AAPL_analysis.csv")

plot_dashboard(AAPL_data, "AAPL")

[*********************100%***********************]  1 of 1 completed

Price            Close  Log_Return  Volatility        RSI
Date                                                     
2025-12-08  277.890015   -0.003198    0.167281  66.456347
2025-12-09  277.179993   -0.002558    0.166387  65.040131
2025-12-10  278.779999    0.005756    0.166714  65.550816
2025-12-11  278.029999   -0.002694    0.150891  68.817878
2025-12-12  278.279999    0.000899    0.148748  62.903841



*   **Hafta 3-4: Portföy Matematiği (Matrix Operations)**
    *   **Görev:** Kullanıcının seçtiği ağırlıklara göre (Örn: %50 THY, %50 Apple) portföyün beklenen getirisini ve varyansını hesaplayan matris çarpımını kodla.
    *   **Çıktı:** Varyans-Kovaryans matrisi ve tek bir risk skoru.

In [ ]:
def portfolio_analysis(symbols,  period='1y'):
    portfolio_data = {}
    for symbol in symbols:
        data = analysis(symbol, period=period)
        portfolio_data[symbol] = data
    return portfolio_data

portfolio = portfolio_analysis(["AAPL", "NVDA", "GARAN.IS"], period='1y')

def expected_returns_of_portfolio(portfolio_data):
    expected_returns = {}
    for symbol, data in portfolio_data.items():
        mean_log_return = data['Log_Return'].mean() * 252  # Annualized Expected Return
        expected_returns[symbol] = mean_log_return
    return expected_returns

return_of_portfolio = expected_returns_of_portfolio(portfolio)
print("Expected Annual Returns of Portfolio:")
for symbol, exp_return in return_of_portfolio.items():
    print(f"{symbol}: {exp_return:.2%}")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Expected Annual Returns of Portfolio:
AAPL: 12.06%
NVDA: 26.87%
GARAN.IS: 14.61%
Portfolio Variances:
AAPL: 0.105281
NVDA: 0.248530
GARAN.IS: 0.147883
